In [ ]:
import struct
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
def parse_ideal_line(path):
    with open(path, "rb") as f:
        version = struct.unpack("<i", f.read(4))[0]
        if version != 7:
            raise ValueError(f"Unsupported spline version: {version}")

        point_count = struct.unpack("<i", f.read(4))[0]
        lap_time = struct.unpack("<i", f.read(4))[0]
        sample_count = struct.unpack("<i", f.read(4))[0]

        # AiPoint: position (vec3), length, id
        points = []
        for _ in range(point_count):
            x, y, z = struct.unpack("<fff", f.read(12))
            length = struct.unpack("<f", f.read(4))[0]
            point_id = struct.unpack("<i", f.read(4))[0]
            points.append([x, y, z, length, point_id])

        extra_count = struct.unpack("<i", f.read(4))[0]
        if extra_count != point_count:
            raise ValueError("Mismatch between point count and extra data count.")

        # AiPointExtra: 18 floats = 72 bytes
        extras = []
        for _ in range(extra_count):
            data = struct.unpack("<" + "f" * 18, f.read(72))
            extras.append(list(data))

    columns = [
        "x", "y", "z", "length", "id",
        "speed", "gas", "brake", "obsolete_lat_g", "radius",
        "side_left", "side_right", "camber", "direction",
        "normal_x", "normal_y", "normal_z",
        "extra_length",
        "forward_x", "forward_y", "forward_z",
        "tag", "grade"
    ]

    df = pd.DataFrame([p + e for p, e in zip(points, extras)], columns=columns)
    return df

In [ ]:
def plot_racing_line_with_map(
    df,
    map_image_path,
    offset=(0, 0),
    scale=1.0,
    color_by="speed",
    title="Racing Line + Map"
):
    # Load map image
    img = Image.open(map_image_path)
    w, h = img.size

    # Create plot
    fig, ax = plt.subplots(figsize=(16, 16), dpi=100)

    # Display image using scaled extent
    extent = [
        offset[0], offset[0] + w * scale,
        offset[1] + h * scale, offset[1]
    ]
    ax.imshow(img, extent=extent, origin="upper", alpha=0.6)

    # Plot racing line
    x = df["x"]
    z = df["z"]

    if color_by in df.columns:
        sc = ax.scatter(x, z, c=df[color_by], cmap="viridis", s=0.25)
        cbar = plt.colorbar(sc, ax=ax)
        cbar.set_label(color_by)
    else:
        ax.plot(x, z, lw=1)

    ax.set_title(f"Racing Line: {title}")
    ax.set_xlabel("x")
    ax.set_ylabel("z")
    ax.set_aspect("equal")
    plt.grid(True)
    plt.show()


In [ ]:
map = "./data/assetto_corsa_tracks/ks_nurburgring/"
versions = {"gp_a":("layout_gp_a/data/ideal_line.ai", "layout_gp_a/map.png"), "gp_b":("layout_gp_b/data/ideal_line.ai", "layout_gp_b/map.png"),
            "sprint_a":("layout_sprint_a/data/ideal_line.ai", "layout_sprint_a/map.png"), "sprint_b": ("layout_sprint_b/data/ideal_line.ai", "layout_sprint_b/map.png")}

name = "sprint_a" # gp_a, gp_b, sprint_a, sprint_b
data_path = map + versions[name][0] 
image_path = map + versions[name][1]
racing_line = parse_ideal_line(data_path)

In [ ]:
racing_line.info()
racing_line.head()

In [ ]:
# Offsets might not align perfectly however its close enough to visually see this code is producing what we desire
# GP Plot Vars 
# scl = 1.2
# off = (-422, -1035)

# Sprint Plot Vars 
scl = 1
off = (-422, -1035)

# Plot 
plot_racing_line_with_map(racing_line, map_image_path=image_path, color_by="speed", title="Racing Line: ks_nurburgring", offset=off, scale=scl)
plot_racing_line_with_map(racing_line, map_image_path=image_path, color_by="side_left", title="Racing Line: ks_nurburgring", offset=off, scale=scl)
plot_racing_line_with_map(racing_line, map_image_path=image_path, color_by="radius", title="Racing Line: ks_nurburgring", offset=off, scale=scl)
plot_racing_line_with_map(racing_line, map_image_path=image_path, color_by="direction", title="Racing Line: ks_nurburgring", offset=off, scale=scl)